In [1]:
import numpy as np
import gurobipy as gp

In [2]:
M = 1000#either set xii travelling from city i to i as 0 or assign high cost value to that decision variable in minimization problem
N = 5#number of cities
obj = np.array([M,132,217,164,58,132,M,290,201,79,217,290,M,113,303,164,201,113,M,196,58,79,303,196,M]+
               [0]*N)#N uis for avoiding loops, need to ponder more i thought it will n-1 in number

In [30]:
A = np.zeros((2*N + (N-1)**2 - (N-1),
              N**2 + N))
b = np.zeros(2*N + (N-1)**2 - (N-1))#Arrival and departure plus no sublooping --> N+N+ ((N-1)^2-(N-1))
direction = np.array(['']*(2*N + (N-1)**2 - (N-1)))

In [31]:
ind_vec = np.array(range(N))#iterator for iterable range
ind_vec

array([0, 1, 2, 3, 4])

In [53]:
ind_vec = np.array(range(N))#Can use broadcasting addition rule array + constant =(5,) operator (1,)=(5,)
row = 0
for j in range(N):
#     A[row,j*N + ind_vec] = 1
    A[row,j*N:(((j+1)*N))] = 1
    b[row] = 1
    direction[row] = '='
    row+=1
# A

In [54]:
ii=1
# A[0,i:5*5-i:5]
A[5,0:N*N-(N-0)+1:N]#go from first city variable till last city pair decision variable in steps of number of cities

# ind_vec*N +i #This is a row slider
# ind_vec +N*i #This is a Column slider

array([1., 1., 1., 1., 1.])

In [55]:
for i in range(N):
    A[row,ind_vec*N +i] = 1
#     A[row,i:N*N-(N-i)+1:N]=-1
    b[row] = 1
    direction[row] = '='
    row+=1
    
A[5:10]

array([[1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

In [56]:
for i in range(1,5):
    for j in range(1,5):
        if i != j:
            A[row,[N**2+i,N**2+j,j*N+i]] = [1,-1,N]
            b[row] = N-1
            direction[row] = '<'
            row+=1

In [57]:
tspMod = gp.Model()
tspMod_x = tspMod.addMVar(N**2 + N,vtype=['B']*25+['C']*5) 
tspMod_con = tspMod.addMConstrs(A, tspMod_x, direction, b)
tspMod.setMObjective(None,obj,0,sense=gp.GRB.MINIMIZE)

tspMod.Params.OutputFlag = 0 
tspMod.optimize()

Restricted license - for non-production use only - expires 2022-01-13


In [58]:
tspMod_x.x

array([-0.,  1., -0., -0., -0., -0., -0.,  1., -0., -0., -0., -0., -0.,
        1., -0., -0., -0., -0., -0.,  1.,  1., -0., -0., -0., -0.,  0.,
        4.,  2.,  1.,  4.])

In [91]:
tspMod.objval
aa=tspMod_x.x[:N*N]
np.where(aa==1)
# tspMod_x.x
# aa

obj[np.where(aa==1)]
np.mod(np.where(aa==1),5)





array([[1, 2, 3, 4, 0]], dtype=int64)

In [92]:
where = 0
miles_traveled = 0
print('Start in city 0.')
for city in range(N):
    where_new = np.where(tspMod_x.x[(where*N):((where+1)*N)])[0][0]#where tells about the city as every 5 based vector column is considered everytime
    dist = obj[where*N+where_new]
    miles_traveled += dist
    print('Then go to city '+str(where_new)+', traveling '+str(dist)+' miles along the way.')
    where = where_new
#     break
print('For a total of '+str(miles_traveled)+' miles.')

Start in city 0.
Then go to city 1, traveling 132 miles along the way.
Then go to city 2, traveling 290 miles along the way.
Then go to city 3, traveling 113 miles along the way.
Then go to city 4, traveling 196 miles along the way.
Then go to city 0, traveling 58 miles along the way.
For a total of 789 miles.
